<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/TOGETHERAI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install together -q

In [3]:
from together import Together
from google.colab import userdata
import os

os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')

client = Together()

response = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
      {
        "role": "user",
        "content": "What are some fun things to do in New York?"
      }
    ]
)
print(response.choices[0].message.content)

### New York City: A Quick‑Start Guide to Fun

Below is a “cheat‑sheet” of things that cater to every mood, budget, and travel style.  Think of it as a menu – pick what sounds good today, or mix and match to create your own adventure.

| Category | Highlight | Why It’s Fun | Rough Cost / Tips |
|----------|-----------|--------------|-------------------|
| **Iconic Landmarks** | Statue of Liberty / Ellis Island | Classic, photos, history | Ferry $25 (includes tickets) |
| | Empire State Building | The view, “The Empire” vibe | 1‑hr: $36 (skip‑the‑line $58) |
| | Top of the Rock | Same view but cheaper, no lines | 1‑hr: $38 |
| | 9/11 Memorial & Museum | Moving tribute | $28 (free on Tuesdays 4‑10 pm) |
| **Broadway & Off‑Broadway** | Ticket‑for‑the‑day (TFTD) at TKTS | Cheap last‑minute deals | $0–$50 depending on show |
| | New Broadway: “O” (Cirque du Soleil) | Spectacular | $120–$200 |
| **Museums & Galleries** | MoMA | Modern art lovers | $25 (free on Friday 5 pm–8 pm) |
| | The Met

In [8]:
# Mixture-of-Agents in 50 lines of code
import asyncio
import os
from together import AsyncTogether, Together

client = Together()
async_client = AsyncTogether()

user_prompt = "What are some fun things to do in SF?"
reference_models = [
    "meta-llama/Llama-3.3-70B-Instruct-Turbo",
]
aggregator_model = "meta-llama/Llama-3.3-70B-Instruct-Turbo"
aggreagator_system_prompt = """You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

Responses from models:"""


async def run_llm(model):
    """Run a single LLM call with a reference model."""
    response = await async_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": user_prompt}],
        temperature=0.7,
        max_tokens=512,
    )
    print(model)
    return response.choices[0].message.content


async def main():
    results = await asyncio.gather(
        *[run_llm(model) for model in reference_models]
    )

    finalStream = client.chat.completions.create(
        model=aggregator_model,
        messages=[
            {"role": "system", "content": aggreagator_system_prompt},
            {
                "role": "user",
                "content": ",".join(str(element) for element in results),
            },
        ],
        stream=True,
    )

    for chunk in finalStream:
        print(chunk.choices[0].delta.content or "", end="", flush=True)


await main()

meta-llama/Llama-3.3-70B-Instruct-Turbo
San Francisco is a city that embodies a unique blend of vibrant culture, rich history, and breathtaking natural beauty, offering countless exciting experiences for visitors. From iconic landmarks to diverse neighborhoods, delectable food scenes, and a wide range of outdoor and cultural activities, there's something for everyone in this eclectic city. Here's a comprehensive guide to help you navigate the best of San Francisco:

### Iconic Landmarks
- **Golden Gate Bridge**: This iconic suspension bridge is not only an engineering marvel but also offers spectacular views of the San Francisco Bay and the city skyline. Visitors can walk, bike, or drive across the bridge for an unforgettable experience.
- **Alcatraz Island**: Once a notorious prison, Alcatraz is now a national park and a must-visit attraction. Take a ferry to the island to explore the cellblock, enjoy the beautifully maintained gardens, and take in the stunning views of the Golden Gat

In [13]:
import os
import time
from concurrent.futures import ThreadPoolExecutor
from together import Together
# Removed: from together.types import ChatCompletionMessageParam # This caused the ImportError

# --- Global Placeholders ---
# These are initialized to None and set up properly in the main block.
client: Together | None = None
TOGETHER_API_KEY: str | None = None

# --- Model Selection (Simulating 4 OSS models for Agents + 1 large OSS model for Combiner) ---
# CHANGED: Using widely available serverless models to resolve Error Code 400
AGENT_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"  # A smaller, faster OSS model for the agents
COMBINER_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # A larger, more powerful OSS model for synthesis

# --- Agent Definitions ---
AGENT_PERSONAS = [
    { "id": 1, "name": "The Historian", "instruction": "Act as a meticulous historian and philosopher. When answering, focus primarily on historical context, evolution, past failures, and long-term societal trends. Provide a comprehensive, academic, and cautious perspective." },
    { "id": 2, "name": "The Technologist", "instruction": "Act as a leading-edge technology developer and futurist. When answering, focus on the current state-of-the-art, potential technological breakthroughs, engineering challenges, and disruptive future impacts. Provide a forward-looking and solution-oriented perspective." },
    { "id": 3, "name": "The Analyst", "instruction": "Act as a cold, objective risk analyst and economist. When answering, focus on quantifiable metrics, financial incentives, regulatory frameworks, immediate risks, and cost-benefit analysis. Provide a pragmatic and data-driven perspective." },
    { "id": 4, "name": "The Creative Writer", "instruction": "Act as a whimsical, speculative fiction author and poet. When answering, focus on abstract concepts, emotional impact, potential utopian or dystopian scenarios, and metaphorical significance. Provide a narrative and imaginative perspective." }
]

COMBINER_INSTRUCTION = (
    "You are the Synthesis Engine, the final layer of the Mixture of Agents (MoA) system. "
    "Your task is to take the original user prompt and the four diverse responses generated by the "
    "expert agents (Historian, Technologist, Analyst, Creative Writer) and fuse them into one single, "
    "cohesive, comprehensive, and high-quality final answer. The final answer must be better than any "
    "individual response by integrating all unique, valuable, and non-redundant insights."
    "The final output must be formatted professionally."
)

def call_together_api(user_query: str, system_prompt: str, model: str) -> str:
    """Uses the globally available Together client to call the chat completions endpoint."""
    global client
    if not client:
        return f"Error: Together AI client not initialized. Check API Key configuration."

    # Standard message format for chat completions
    messages: list[dict] = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.5,
            max_tokens=1024,
        )

        text = response.choices[0].message.content
        return text if text is not None else 'No response generated.'

    except Exception as e:
        # Include the specific error message for better debugging if a new error occurs
        return f"Together AI API Request Failed: {e}"

def run_agent(agent, user_prompt):
    """Runs a single agent against the user prompt using the Agent model."""
    print(f"  -> Running Agent {agent['id']}: {agent['name']} ({AGENT_MODEL.split('/')[-1]})...")
    response = call_together_api(user_prompt, agent['instruction'], AGENT_MODEL)
    return (agent['name'], response)

def run_moa_simulation(user_prompt):
    """
    Executes the two-stage MoA process:
    1. Concurrent Agent Execution (using ThreadPoolExecutor).
    2. Combiner Synthesis.
    """
    if not user_prompt.strip():
        return "Please provide a prompt to run the MoA simulation."

    # Check if the client was successfully initialized in the main block
    if not client:
        print("\n==================================================")
        print("||  TOGETHER API KEY REQUIRED: The client is not initialized. Please ensure the 'TOGETHER_API_KEY' environment variable is set. ||")
        print("==================================================")
        return

    print(f"\n--- Starting Together MoA Simulation for Prompt ---\n'{user_prompt}'\n")

    # --- Step 1: Agent Execution (Concurrent/Parallel) ---
    agent_responses = {}

    print("Executing Agents concurrently using ThreadPoolExecutor...")
    start_time = time.time()

    with ThreadPoolExecutor(max_workers=len(AGENT_PERSONAS)) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(run_agent, agent, user_prompt) for agent in AGENT_PERSONAS]

        # Collect results as they complete
        for future in futures:
            agent_name, response_text = future.result()
            agent_responses[agent_name] = response_text
            print(f"  <- Agent {agent_name} complete.")

    agent_time = time.time() - start_time
    print(f"\n--- Agent Responses Collected in {agent_time:.2f}s ---")

    for name, response in agent_responses.items():
        # Check if response failed before slicing
        if response.startswith("Together AI API Request Failed"):
            print(f"\n[ {name} ]\n{'-'*len(name)}\n{response}\n")
        else:
            print(f"\n[ {name} ]\n{'-'*len(name)}\n{response[:150]}...\n")

    # --- Step 2: Combiner Synthesis ---
    print(f"Executing Combiner Agent for Synthesis ({COMBINER_MODEL.split('/')[-1]})...")
    combiner_start_time = time.time()

    # Format the collected responses for the Combiner Agent
    synthesis_prompt = (
        f"Original User Prompt: {user_prompt}\n\n"
        f"--- Agent Inputs for Synthesis ---\n\n"
        f"1. Historian's Response:\n---\n{agent_responses.get('The Historian', 'N/A')}\n---\n\n"
        f"2. Technologist's Response:\n---\n{agent_responses.get('The Technologist', 'N/A')}\n---\n\n"
        f"3. Analyst's Response:\n---\n{agent_responses.get('The Analyst', 'N/A')}\n---\n\n"
        f"4. Creative Writer's Response:\n---\n{agent_responses.get('The Creative Writer', 'N/A')}\n---\n\n"
        f"Please generate the single, best, synthesized final answer. Ensure the final output is well-structured and comprehensive."
    )

    # The combiner uses the larger, more capable model for superior synthesis
    final_synthesis = call_together_api(synthesis_prompt, COMBINER_INSTRUCTION, COMBINER_MODEL)

    combiner_time = time.time() - combiner_start_time
    print(f"--- Synthesis Complete in {combiner_time:.2f}s ---")

    print("\n==================================================")
    print("||           FINAL TOGETHER MOA SYNTHESIS RESULT           ||")
    print("==================================================")
    print(final_synthesis)
    print("==================================================")
    print(f"Total simulated MoA time (including concurrent execution): {agent_time + combiner_time:.2f} seconds.")

    return final_synthesis

if __name__ == "__main__":


    # We are using os.environ.get here to retrieve the key set above (or from a pre-set environment).
    TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

    # --- 2. Initialize the Together Client ---
    if TOGETHER_API_KEY:
        try:
            client = Together(api_key=TOGETHER_API_KEY)
            print("Together AI Client initialized successfully.")
        except Exception as e:
            print(f"Error initializing Together client: {e}")
            client = None
    else:
        print("TOGETHER_API_KEY environment variable not found. Please ensure the key is set (e.g., using the userdata method if applicable).")
        client = None

    # --- 3. Run the simulation ---
    example_prompt = "What are the long-term societal and economic impacts of fusion energy development?"
    run_moa_simulation(example_prompt)


Together AI Client initialized successfully.

--- Starting Together MoA Simulation for Prompt ---
'What are the long-term societal and economic impacts of fusion energy development?'

Executing Agents concurrently using ThreadPoolExecutor...
  -> Running Agent 1: The Historian (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 2: The Technologist (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 3: The Analyst (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 4: The Creative Writer (Mistral-7B-Instruct-v0.2)...
  <- Agent The Historian complete.
  <- Agent The Technologist complete.
  <- Agent The Analyst complete.
  <- Agent The Creative Writer complete.

--- Agent Responses Collected in 4.50s ---

[ The Historian ]
-------------
 The development of fusion energy, a process that mimics the sun's nuclear fusion reactions to produce vast amounts of power, has the potential to rev...


[ The Technologist ]
----------------
 Fusion energy development, if successfully achieved on a commercial 

In [14]:
import os
import time
from concurrent.futures import ThreadPoolExecutor
from together import Together  # Requires: pip install together

# Global Placeholders
client: Together | None = None
TOGETHER_API_KEY: str | None = None

# Model Selection
AGENT_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"  # Lightweight model for agents
COMBINER_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"  # Powerful model for synthesis

# Agent Personas tailored for Cryptocurrency
AGENT_PERSONAS = [
    {
        "id": 1,
        "name": "The Crypto Historian",
        "instruction": "Act as a meticulous historian of cryptocurrency and blockchain. When answering, focus primarily on historical context, key events like Bitcoin's creation, major crashes (e.g., 2018 bear market), regulatory milestones, and long-term societal trends in adoption. Provide a comprehensive, academic, and cautious perspective."
    },
    {
        "id": 2,
        "name": "The Blockchain Technologist",
        "instruction": "Act as a leading-edge blockchain developer and futurist. When answering, focus on the current state-of-the-art in cryptocurrency tech, potential breakthroughs like Ethereum upgrades or layer-2 scaling, engineering challenges such as scalability and security, and disruptive future impacts on finance. Provide a forward-looking and solution-oriented perspective."
    },
    {
        "id": 3,
        "name": "The Market Analyst",
        "instruction": "Act as a cold, objective cryptocurrency market analyst and economist. When answering, focus on quantifiable metrics like market cap, volatility indices, financial incentives for adoption, regulatory frameworks, immediate risks (e.g., hacks, rug pulls), and cost-benefit analysis of investments. Provide a pragmatic and data-driven perspective."
    },
    {
        "id": 4,
        "name": "The Crypto Visionary",
        "instruction": "Act as a whimsical, speculative fiction author and poet focused on cryptocurrency. When answering, focus on abstract concepts like decentralization's promise, emotional impact on financial freedom, potential utopian (e.g., borderless economies) or dystopian (e.g., cyberpunk surveillance states) scenarios, and metaphorical significance of digital gold. Provide a narrative and imaginative perspective."
    }
]

COMBINER_INSTRUCTION = (
    "You are the Synthesis Engine, the final layer of the Mixture of Agents (MoA) system for cryptocurrency analysis. "
    "Your task is to take the original user prompt and the four diverse responses generated by the "
    "expert agents (Crypto Historian, Blockchain Technologist, Market Analyst, Crypto Visionary) and fuse them into one single, "
    "cohesive, comprehensive, and high-quality final answer. The final answer must be better than any "
    "individual response by integrating all unique, valuable, and non-redundant insights. "
    "Structure the output professionally, with sections for historical context, technological outlook, economic analysis, speculative futures, and overall synthesis. "
    "Ensure the response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability."
)

def call_together_api(user_query: str, system_prompt: str, model: str) -> str:
    """Uses the globally available Together client to call the chat completions endpoint."""
    global client
    if not client:
        return f"Error: Together AI client not initialized. Check API Key configuration."

    # Standard message format for chat completions
    messages: list[dict] = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.5,
            max_tokens=1024,
        )

        text = response.choices[0].message.content
        return text if text is not None else 'No response generated.'

    except Exception as e:
        # Include the specific error message for better debugging if a new error occurs
        return f"Together AI API Request Failed: {e}"

def run_agent(agent, user_prompt):
    """Runs a single agent against the user prompt using the Agent model."""
    print(f"  -> Running Agent {agent['id']}: {agent['name']} ({AGENT_MODEL.split('/')[-1]})...")
    response = call_together_api(user_prompt, agent['instruction'], AGENT_MODEL)
    return (agent['name'], response)

def run_moa_simulation(user_prompt):
    """
    Executes the two-stage MoA process for cryptocurrency analysis:
    1. Concurrent Agent Execution (using ThreadPoolExecutor).
    2. Combiner Synthesis.
    """
    if not user_prompt.strip():
        return "Please provide a prompt to run the MoA simulation."

    # Check if the client was successfully initialized in the main block
    if not client:
        print("\n==================================================")
        print("||  TOGETHER API KEY REQUIRED: The client is not initialized. Please ensure the 'TOGETHER_API_KEY' environment variable is set. ||")
        print("==================================================")
        return

    print(f"\n--- Starting Crypto MoA Simulation for Prompt ---")
    print(f"'{user_prompt}'")
    print("\n")

    # --- Step 1: Agent Execution (Concurrent/Parallel) ---
    agent_responses = {}

    print("Executing Crypto Agents concurrently using ThreadPoolExecutor...")
    start_time = time.time()

    with ThreadPoolExecutor(max_workers=len(AGENT_PERSONAS)) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(run_agent, agent, user_prompt) for agent in AGENT_PERSONAS]

        # Collect results as they complete
        for future in futures:
            agent_name, response_text = future.result()
            agent_responses[agent_name] = response_text
            print(f"  <- Agent {agent_name} complete.")

    agent_time = time.time() - start_time
    print(f"\n--- Agent Responses Collected in {agent_time:.2f}s ---")

    for name, response in agent_responses.items():
        # Check if response failed before slicing
        if response.startswith("Together AI API Request Failed"):
            print(f"\n[ {name} ]\n{'-'*len(name)}\n{response}\n")
        else:
            print(f"\n[ {name} ]\n{'-'*len(name)}\n{response[:150]}...\n")

    # --- Step 2: Combiner Synthesis ---
    print(f"Executing Combiner Agent for Synthesis ({COMBINER_MODEL.split('/')[-1]})...")
    combiner_start_time = time.time()

    # Format the collected responses for the Combiner Agent
    synthesis_prompt = (
        f"Original User Prompt: {user_prompt}\n\n"
        f"--- Agent Inputs for Synthesis ---\n\n"
        f"1. Crypto Historian's Response:\n---\n{agent_responses.get('The Crypto Historian', 'N/A')}\n---\n\n"
        f"2. Blockchain Technologist's Response:\n---\n{agent_responses.get('The Blockchain Technologist', 'N/A')}\n---\n\n"
        f"3. Market Analyst's Response:\n---\n{agent_responses.get('The Market Analyst', 'N/A')}\n---\n\n"
        f"4. Crypto Visionary's Response:\n---\n{agent_responses.get('The Crypto Visionary', 'N/A')}\n---\n\n"
        f"Please generate the single, best, synthesized final answer. Ensure the final output is well-structured and comprehensive."
    )

    # The combiner uses the larger, more capable model for superior synthesis
    final_synthesis = call_together_api(synthesis_prompt, COMBINER_INSTRUCTION, COMBINER_MODEL)

    combiner_time = time.time() - combiner_start_time
    print(f"--- Synthesis Complete in {combiner_time:.2f}s ---")

    print("\n==================================================")
    print("||           FINAL CRYPTO MOA SYNTHESIS RESULT           ||")
    print("==================================================")
    print(final_synthesis)
    print("==================================================")
    print(f"Total simulated MoA time (including concurrent execution): {agent_time + combiner_time:.2f} seconds.")

    return final_synthesis

if __name__ == "__main__":
    # Retrieve API Key (set via environment variable or userdata in Colab)
    TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

    # Initialize the Together Client
    if TOGETHER_API_KEY:
        try:
            client = Together(api_key=TOGETHER_API_KEY)
            print("Together AI Client initialized successfully.")
        except Exception as e:
            print(f"Error initializing Together client: {e}")
            client = None
    else:
        print("TOGETHER_API_KEY environment variable not found. Please ensure the key is set.")
        client = None

    # Example Prompt for Cryptocurrency Analysis
    example_prompt = "What are the long-term societal and economic impacts of widespread cryptocurrency adoption?"
    run_moa_simulation(example_prompt)

Together AI Client initialized successfully.

--- Starting Crypto MoA Simulation for Prompt ---
'What are the long-term societal and economic impacts of widespread cryptocurrency adoption?'


Executing Crypto Agents concurrently using ThreadPoolExecutor...
  -> Running Agent 1: The Crypto Historian (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 2: The Blockchain Technologist (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 3: The Market Analyst (Mistral-7B-Instruct-v0.2)...
  -> Running Agent 4: The Crypto Visionary (Mistral-7B-Instruct-v0.2)...
  <- Agent The Crypto Historian complete.
  <- Agent The Blockchain Technologist complete.
  <- Agent The Market Analyst complete.
  <- Agent The Crypto Visionary complete.

--- Agent Responses Collected in 4.67s ---

[ The Crypto Historian ]
--------------------
 The long-term societal and economic impacts of widespread cryptocurrency adoption are both profound and complex, with potential benefits and challeng...


[ The Blockchain Technolog